In [ ]:
from transformers import pipeline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import tqdm

In [ ]:
device = 'mps'

In [ ]:
def batch_encoding_to_device(batch):
    return {k: v.to(device) for k, v in batch.items()}

## 5. Datasets

### 5.1 What if my dataset isn't on the Hub

In [ ]:
from datasets import load_dataset

In [ ]:
squad_it_dataset = load_dataset('json', data_files='data/SQuAD_it-train.json', field='data')

In [ ]:
squad_it_dataset

In [ ]:
squad_it_dataset['train'][0]['paragraphs'][0]

In [ ]:
data_files = {'train': 'data/SQuAD_it-train.json', 'test': 'data/SQuAD_it-test.json'}

In [ ]:
squad_it_dataset = load_dataset('json', data_files=data_files, field='data')

In [ ]:
squad_it_dataset

In [ ]:
# Automatically unzip
#data_files = {"train": "data/SQuAD_it-train.json.gz", "test": "data/SQuAD_it-test.json.gz"}
#squad_it_dataset = load_dataset("json", data_files=data_files, field="data")

In [ ]:
url = "https://github.com/crux82/squad-it/raw/master/"
data_files = {
    "train": url + "SQuAD_it-train.json.gz",
    "test": url + "SQuAD_it-test.json.gz",
}
squad_it_dataset = load_dataset("json", data_files=data_files, field="data")

In [ ]:
squad_it_dataset

### 5.2 Time to slice and dice

In [ ]:
from datasets import load_dataset

In [ ]:
data_files = {'train': 'data/drugsComTrain_raw.tsv', 'test': 'data/drugsComTest_raw.tsv'}

In [ ]:
drug_dataset = load_dataset('csv', data_files=data_files, delimiter='\t')

In [ ]:
drug_dataset

In [ ]:
drug_sample = drug_dataset['train'].shuffle(seed=42).select(range(1000))

In [ ]:
drug_sample[:3]

In [ ]:
for split in drug_dataset.keys():
    assert len(drug_dataset[split]) == len(drug_dataset[split].unique('Unnamed: 0'))

In [ ]:
drug_dataset = drug_dataset.rename_column(
    original_column_name='Unnamed: 0', new_column_name='patient_id'
)

In [ ]:
drug_dataset

In [ ]:
def lowercase_condition(example):
    return {'condition': example['condition'].lower()}

In [ ]:
# Raises attribute error since some samples are None
#drug_dataset.map(lowercase_condition)

In [ ]:
drug_dataset = drug_dataset.filter(lambda x: x['condition'] is not None)

In [ ]:
drug_dataset = drug_dataset.map(lowercase_condition)

In [ ]:
drug_dataset['train']['condition'][:3]

In [ ]:
def compute_review_length(example):
    return {'review_length': len(example['review'].split())}

In [ ]:
drug_dataset = drug_dataset.map(compute_review_length)

In [ ]:
drug_dataset['train'][0]

In [ ]:
drug_dataset['train'].sort('review_length')[:3]

In [ ]:
drug_dataset = drug_dataset.filter(lambda x: x['review_length'] > 30)

In [ ]:
drug_dataset.num_rows

In [ ]:
sorted(drug_dataset['train']['review_length'])[-3:]

In [ ]:
import html

In [ ]:
text = "I&#039;m a transformer called BERT"

In [ ]:
html.unescape(text)

In [ ]:
drug_dataset = drug_dataset.map(lambda x: {'review': html.unescape(x['review'])})

In [ ]:
new_drug_dataset = drug_dataset.map(
    lambda x: {'review': [html.unescape(o) for o in x['review']]}, batched=True
)

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['review'], truncation=True)

In [ ]:
%time tokenized_data = drug_dataset.map(tokenize_function, batched=True)

In [ ]:
%time tokenized_data = drug_dataset.map(tokenize_function, batched=False)

In [ ]:
%time tokenized_data = drug_dataset.map(tokenize_function, batched=True, num_proc=10)

In [ ]:
def tokenize_and_split(examples):
    return tokenizer(examples['review'], truncation=True, max_length=128, return_overflowing_tokens=True)

In [ ]:
result = tokenize_and_split(drug_dataset['train'][0])

In [ ]:
[len(inp) for inp in result['input_ids']]

In [ ]:
# Raises arrow invalid for mismatched column lengths 
#tokenized_dataset = drug_dataset.map(tokenize_and_split, batched=True)

In [ ]:
tokenized_dataset = drug_dataset.map(
    tokenize_and_split, batched=True, remove_columns=drug_dataset['train'].column_names
)

In [ ]:
len(tokenized_dataset['train']), len(drug_dataset['train'])

In [ ]:
drug_dataset['train'].features

In [ ]:
tokenized_dataset['train'].features

In [ ]:
def tokenize_and_split(examples):
    result = tokenizer(
        examples['review'],
        truncation=True,
        max_length=128,
        return_overflowing_tokens=True,
    )
    # Extract mapping between new and old indices
    sample_map = result.pop('overflow_to_sample_mapping')
    for key, values in examples.items():
        result[key] = [values[i] for i in sample_map]
    return result

In [ ]:
tokenized_dataset = drug_dataset.map(tokenize_and_split, batched=True)

In [ ]:
tokenized_dataset

In [ ]:
print(tokenized_dataset['train'][:2])

In [ ]:
drug_dataset.set_format('pandas')

In [ ]:
drug_dataset['train'][:3]

In [ ]:
train_df = drug_dataset['train'][:]

In [ ]:
frequencies = (
    train_df["condition"]
    .value_counts()
    .to_frame()
    .reset_index()
    .rename(columns={"index": "condition", "condition": "frequency"})
)

In [ ]:
frequencies.head()

In [ ]:
from datasets import Dataset

In [ ]:
freq_dataset = Dataset.from_pandas(frequencies)

In [ ]:
freq_dataset

In [ ]:
drug_mean_rating = (
    train_df[['drugName', 'rating']]
    .groupby(['drugName'])
    .mean()
    .sort_values(by='rating')
    .reset_index()
    .rename(columns={'rating':'mean rating'})
)

In [ ]:
drug_mean_rating.tail()

In [ ]:
drug_dataset.reset_format()

In [ ]:
drug_dataset['train'][:3]

In [ ]:
drug_dataset_clean = drug_dataset['train'].train_test_split(train_size=0.8, seed=42)

In [ ]:
drug_dataset_clean['val'] = drug_dataset_clean.pop('test')

In [ ]:
drug_dataset_clean['test'] = drug_dataset['test']

In [ ]:
drug_dataset_clean

In [ ]:
drug_dataset_clean.save_to_disk('data/drug-reviews')

In [ ]:
from datasets import load_from_disk

In [ ]:
drug_dataset_reloaded = load_from_disk('data/drug-reviews')

In [ ]:
for split, dataset in drug_dataset_clean.items():
    dataset.to_json(f'data/drug-reviews-{split}.jsonl')

In [ ]:
data_files = {
    'train': 'data/drug-reviews-train.jsonl',
    'val': 'data/drug-reviews-val.jsonl',
    'test': 'data/drug-reviews-test.jsonl',
}

In [ ]:
drug_dataset_reloaded = load_dataset('json', data_files=data_files)

In [ ]:
drug_dataset_reloaded

### 5.3 Big data

In [ ]:
from datasets import load_dataset, DownloadConfig

In [ ]:
data_files = "https://huggingface.co/datasets/casinca/PUBMED_title_abstracts_2019_baseline/resolve/main/PUBMED_title_abstracts_2019_baseline.jsonl.zst"
pubmed_dataset = load_dataset(
    "json",
    data_files=data_files,
    split="train",
    download_config=DownloadConfig(delete_extracted=True),  # (optional arg)using DownloadConfig to save HD space
)

In [ ]:
pubmed_dataset

In [ ]:
print (pubmed_dataset[10000000]['text'])

In [ ]:
import psutil

In [ ]:
f'RAM used: {psutil.Process().memory_info().rss / 1024 ** 2 :.2f} MB'

In [ ]:
n_files = pubmed_dataset.dataset_size
print (f'Number of files in dataset : {n_files}')
size_gb = n_files / 1024 ** 3
print (f'Dataset size (cache file) : {size_gb:.2f}')

In [ ]:
import timeit

code_snippet = """batch_size = 1000

for idx in range(0, len(pubmed_dataset), batch_size):
    _ = pubmed_dataset[idx:idx + batch_size]
"""

time = timeit.timeit(stmt=code_snippet, number=1, globals=globals())
print(
    f"Iterated over {len(pubmed_dataset)} examples (about {size_gb:.1f} GB) in "
    f"{time:.1f}s, i.e. {size_gb/time:.3f} GB/s"
)

In [ ]:
pubmed_dataset_streamed = load_dataset(
    "json",
    data_files=data_files,
    split="train",
    download_config=DownloadConfig(delete_extracted=True),  # (optional arg)using DownloadConfig to save HD space
    streaming=True,
)

In [ ]:
pubmed_dataset_streamed

In [ ]:
next(iter(pubmed_dataset_streamed))

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
tokenized_dataset = pubmed_dataset_streamed.map(lambda x: tokenizer(x['text']))

In [ ]:
tokenized_dataset

In [ ]:
print(next(iter(tokenized_dataset)))

In [ ]:
shuffled_dataset = pubmed_dataset_streamed.shuffle(buffer_size=10000, seed=42)

In [ ]:
print(next(iter(shuffled_dataset)))

In [ ]:
dataset_head = pubmed_dataset_streamed.take(5)

In [ ]:
list(dataset_head)

In [ ]:
train_dataset = shuffled_dataset.skip(1000)
val_dataset = shuffled_dataset.take(1000)

In [ ]:
from itertools import islice
from datasets import interleave_datasets

In [ ]:
combined_dataset = interleave_datasets([val_dataset, val_dataset])

In [ ]:
list(islice(combined_dataset, 2))

### 5.4 Creating your own dataset

In [ ]:
import requests

In [ ]:
url = "https://api.github.com/repos/huggingface/datasets/issues?page=1&per_page=1"
response = requests.get(url)

In [ ]:
response.status_code

In [ ]:
response.json()

In [ ]:
#github_token = 'create personal access token on github'
#headers = {'Authorization': f'token {github_token}'}

In [ ]:
import time
import math
from pathlib import Path

def fetch_issues(
    owner="huggingface",
    repo="datasets",
    num_issues=10_000,
    rate_limit=5_000,
    issues_path=Path("."),
):
    if not issues_path.is_dir():
        issues_path.mkdir(exist_ok=True)

    batch = []
    all_issues = []
    per_page = 100  # Number of issues to return per page
    num_pages = math.ceil(num_issues / per_page)
    base_url = "https://api.github.com/repos"

    for page in tqdm.tqdm(range(num_pages)):
        # Query with state=all to get both open and closed issues
        query = f"issues?page={page}&per_page={per_page}&state=all"
        issues = requests.get(f"{base_url}/{owner}/{repo}/{query}", headers=headers)
        batch.extend(issues.json())

        if len(batch) > rate_limit and len(all_issues) < num_issues:
            all_issues.extend(batch)
            batch = []  # Flush batch for next time period
            print(f"Reached GitHub rate limit. Sleeping for one hour ...")
            time.sleep(60 * 60 + 1)

    all_issues.extend(batch)
    df = pd.DataFrame.from_records(all_issues)
    df.to_json(f"{issues_path}/{repo}-issues.jsonl", orient="records", lines=True)
    print(
        f"Downloaded all the issues for {repo}! Dataset stored at {issues_path}/{repo}-issues.jsonl"
    )

In [ ]:
fetch_issues(num_issues=3000, issues_path=Path('data'))

In [ ]:
from datasets import load_dataset

In [ ]:
issues_dataset = load_dataset("json", data_files="data/datasets-issues.jsonl", split="train")

In [ ]:
issues_dataset

In [ ]:
sample = issues_dataset.shuffle(seed=42).select(range(10))

In [ ]:
for url, pr in zip(sample['html_url'], sample['pull_request']):
    print (f'>> URL: {url}')
    print (f'>> PR: {pr}\n')

In [ ]:
issues_dataset = issues_dataset.map(
    lambda x: {'is_pull_request': x['pull_request'] is not None}
)

In [ ]:
issues_dataset['is_pull_request'][:6]

In [ ]:
issues_dataset['html_url'][:6]

In [ ]:
issues_dataset_df = issues_dataset.filter(lambda x: x['is_pull_request'])

In [ ]:
issues_dataset_df.set_format('pandas')

In [ ]:
issues_dataset_df = issues_dataset_df[:]

In [ ]:
(issues_dataset_df['closed_at'] - issues_dataset_df['created_at']).mean()

In [ ]:
issue_number = 2792
url = f"https://api.github.com/repos/huggingface/datasets/issues/{issue_number}/comments"
response = requests.get(url, headers=headers)
response.json()

In [ ]:
def get_comments(issue_number):
    url = f"https://api.github.com/repos/huggingface/datasets/issues/{issue_number}/comments"
    response = requests.get(url, headers=headers)
    return [r["body"] for r in response.json()]

In [ ]:
get_comments(issue_number)

In [ ]:
issues_with_comments_dataset = issues_dataset.map(
    lambda x: {'comments': get_comments(x['number'])}
)

In [ ]:
issues_with_comments_dataset.to_json('data/datasets-issues-comments.jsonl')

### 5.5 Semantic search with FAISS

In [ ]:
from datasets import load_dataset

In [ ]:
# specify train so it returns a dataset instead of datasetdict
#issues_dataset = load_dataset("json", data_files="data/datasets-issues-comments.jsonl", split="train")

In [ ]:
issues_dataset = load_dataset("lewtun/github-issues", split="train")

In [ ]:
issues_dataset

In [ ]:
issues_dataset = issues_dataset.filter(
    lambda x: (x['is_pull_request'] == False and len(x['comments']) > 0)
)

In [ ]:
issues_dataset

In [ ]:
columns = issues_dataset.column_names

In [ ]:
columns_to_keep = ['title', 'body', 'html_url', 'comments']

In [ ]:
columns_to_remove = set(columns_to_keep).symmetric_difference(columns)

In [ ]:
issues_dataset = issues_dataset.remove_columns(columns_to_remove)

In [ ]:
issues_dataset.set_format('pandas')

In [ ]:
df = issues_dataset[:]

In [ ]:
df['comments'][0].tolist()

In [ ]:
comments_df = df.explode('comments', ignore_index=True)

In [ ]:
comments_df.head()

In [ ]:
from datasets import Dataset

In [ ]:
# can also make a dataset from dict
comments_dataset = Dataset.from_pandas(comments_df)

In [ ]:
comments_dataset

In [ ]:
# Explode without using pandas
#issues_dataset.reset_format()

# Example function to explode the comments
def explode_comments(batch):
    new_batch = {k: [] for k in batch}  # Initialize a new batch dictionary
    for i in range(len(batch['comments'])):
        for comment in batch['comments'][i]:
            for key in batch:
                if key == 'comments':
                    new_batch[key].append(comment)
                else:
                    new_batch[key].append(batch[key][i])
    return new_batch

#batch = issues_dataset[0:1]
#exploded_batch = explode_comments(batch)

# Use map with batched=True and keep the structure consistent
#exploded_dataset = issues_dataset.map(
#    explode_comments,
#    batched=True,  # Process examples in batches
#)

In [ ]:
comments_dataset = comments_dataset.map(
    lambda x: {'comment_length': len(x['comments'].split())}
)

In [ ]:
comments_dataset = comments_dataset.filter(
    lambda x: x['comment_length'] > 15
)

In [ ]:
comments_dataset

In [ ]:
def concatenate_text(examples):
    return {
        'text': examples['title']
        + ' \n '
        + examples['body']
        + ' \n '
        + examples['comments']
    }

In [ ]:
comments_dataset = comments_dataset.map(concatenate_text)

In [ ]:
from transformers import AutoTokenizer, AutoModel

In [ ]:
#ckeckpoint = 'sentence-transformers/multi-qa-mpnet-base-dot-v1'
checkpoint = 'sentence-transformers/multi-qa-distilbert-cos-v1'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
model = AutoModel.from_pretrained(checkpoint, device_map=device)

In [ ]:
model.device

In [ ]:
total = 0
for p in model.parameters():
    total += p.numel()
np.log10(total)

In [ ]:
# for mpnet
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors='pt'
    )
    encoded_input = batch_encoding_to_device(encoded_input)
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

- unsqueeze: add dimension
- expand: create copies
- clamp: min/max clip

In [ ]:
# for distilbert
import torch
import torch.nn.functional as F

# take mean of all tokens
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    mean_pool = (
        torch.sum(token_embeddings * input_mask_expanded, 1) /
        torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    )
    return mean_pool

def get_embeddings(text_list):
    # tokenize sentences
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors='pt'
    )
    encoded_input = encoded_input.to(device)

    # compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input, return_dict=True)

    # perform pooling
    embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # normalize embeddings
    embeddings = F.normalize(embeddings, p=2, dim=1)

    return embeddings

In [ ]:
embeddings = get_embeddings(comments_dataset['text'][0])

In [ ]:
embeddings.shape

In [ ]:
embeddings_dataset = comments_dataset.map(
    lambda x: {'embeddings': get_embeddings(x['text']).detach().cpu().numpy()[0]}
)

In [ ]:
embeddings_dataset

In [ ]:
embeddings_dataset.add_faiss_index(column='embeddings')

In [ ]:
question = 'How can I load a dataset offline?'

In [ ]:
question_embedding = get_embeddings([question]).detach().cpu().numpy()

In [ ]:
question_embedding.shape

In [ ]:
scores, samples = embeddings_dataset.get_nearest_examples(
    'embeddings', question_embedding, k=5
)

In [ ]:
samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

In [ ]:
#for _, row in samples_df.iterrows():
#    print(f"COMMENT: {row.comments}")
#    print(f"SCORE: {row.scores}")
#    print(f"TITLE: {row.title}")
#    print(f"URL: {row.html_url}")
#    print("=" * 50)
#    print()

#### Using dot product

In [ ]:
query_emb = torch.Tensor(question_embedding).to(device)

In [ ]:
query_emb.shape

In [ ]:
doc_emb = torch.Tensor(embeddings_dataset['embeddings']).to(device)

In [ ]:
doc_emb.shape

In [ ]:
scores = torch.mm(query_emb, doc_emb.transpose(0, 1))[0].cpu().tolist()

In [ ]:
doc_score_pairs = list(zip(embeddings_dataset['text'], scores))

In [ ]:
doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)

In [ ]:
for doc, score in doc_score_pairs[:5]:
    print(f"DOC: {doc}")
    print(f"SCORE: {score}")
    print("=" * 50)
    print()